In [ ]:
# This code defines a Pydantic model and provides methods to convert the model to YAML format,
# including field descriptions as comments, and to load the model from a YAML file.

In [ ]:
from pydantic import BaseModel, Field
import yaml

class MyModel(BaseModel):
    name: str = Field(description="The name of the item")
    count: int = Field(description="Number of items")
    active: bool = Field(description="Whether the item is active")
    
    def to_yaml(self):
        # Get the model's JSON schema, which includes field descriptions
        schema = self.model_json_schema()
        
        # Create a dictionary with values and their descriptions
        output = {}
        for field_name, field_value in self.model_dump().items():
            field_info = schema["properties"].get(field_name, {})
            description = field_info.get("description", "")
            
            # Add to output with description as a comment
            if description:
                output[f"{field_name} # {description}"] = field_value
            else:
                output[field_name] = field_value
        
        # Convert to YAML
        return yaml.dump(output, sort_keys=False)
    
    def to_yaml_file(self, file_path):
        with open(file_path, 'w') as file:
            file.write(self.to_yaml())
    
    @classmethod
    def from_yaml(cls, yaml_str):
        data = yaml.safe_load(yaml_str)
        # Remove comments (if any) from the keys
        data = {k.split(' # ')[0]: v for k, v in data.items()}
        return cls(**data)
    
    @classmethod
    def from_yaml_file(cls, file_path):
        with open(file_path, 'r') as file:
            return cls.from_yaml(file.read())

In [ ]:
# Example usage
item = MyModel(name="test", count=5, active=True)
print(item)
print(item.to_yaml())
item.to_yaml_file('output.yaml')
loaded = MyModel.from_yaml_file('output.yaml')

In [ ]:
assert loaded == item